In [ ]:
import requests
import json
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=IBM&apikey=<APIKEY>'
r = requests.get(url)
data = r.json()
with open("market_data.json", "w") as f:
    json.dump(data, f, indent=4)

In [ ]:
import polars as pl
import json

with open("market_data.json", "r") as f:
    json_data = json.load(f)

# Step 1: Extract "Monthly Time Series"
time_series = json_data["Monthly Time Series"]

# Step 2: Convert to list of dicts with date as a column
records = []
for date, values in time_series.items():
    record = {"date": date}
    record.update({
        "open": float(values["1. open"]),
        "high": float(values["2. high"]),
        "low": float(values["3. low"]),
        "close": float(values["4. close"]),
        "volume": int(values["5. volume"])
    })
    records.append(record)

# Step 3: Create Polars DataFrame
df = pl.DataFrame(records)
print(df.head())

In [ ]:
# Inspect schema
print(df.schema)  # Shows column names and their data types

# Alternatively, just view data types
print("dtypes" , df.dtypes)


In [ ]:

# Convert 'date' to datetime and financial columns to appropriate numeric types
df = df.with_columns([
    pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date"),
    pl.col("open").cast(pl.Float64),
    pl.col("high").cast(pl.Float64),
    pl.col("low").cast(pl.Float64),
    pl.col("close").cast(pl.Float64),
    pl.col("volume").cast(pl.Int64)
])
print(df.schema)

In [ ]:
# Correct method: combine all predicates with AND
df_clean = df.filter(
    pl.fold(
        acc=pl.lit(True),
        function=lambda acc, col: acc & col.is_not_null(),
        exprs=df.columns
    )
)

In [ ]:
# Remove duplicates based on date
df_clean = df_clean.unique(subset=["date"])


In [ ]:
# Sort by date first
df_clean = df_clean.sort("date")

# Calculate monthly return
df_clean = df_clean.with_columns([
    (pl.col("close") / pl.col("close").shift(1) - 1).alias("monthly_return"),
    pl.col("close").rolling_mean(window_size=3).alias("3M_MA"),
    pl.col("close").rolling_std(window_size=3).alias("3M_volatility")
])

# Sort by date first
df_clean = df_clean.sort("date")

# Calculate monthly return
df_clean = df_clean.with_columns([
    (pl.col("close") / pl.col("close").shift(1) - 1).alias("monthly_return"),
    pl.col("close").rolling_mean(window_size=3).alias("3M_MA"),
    pl.col("close").rolling_std(window_size=3).alias("3M_volatility")
])
print(df_clean.head())

In [ ]:
import altair as alt
price_chart = alt.Chart(df_clean).mark_line().encode(
    x='date:T',
    y='close',
    tooltip=['date:T', 'close']
).properties(title='IBM Monthly Closing Price')

volume_chart = alt.Chart(df_clean).mark_area(opacity=0.3).encode(
    x='date:T',
    y='volume'
).properties(title='IBM Monthly Volume')


In [ ]:
price_chart

In [ ]:
volume_chart

In [ ]:
# Assuming `df_clean` has 'moving_avg' column
trend_chart = alt.Chart(df_clean).mark_line().encode(
    x='date:T',
    y='moving_avg',
    color=alt.value('orange')
).properties(title='30-Month Moving Average')

# Overlay with actual close prices
combined = price_chart + trend_chart


In [ ]:
combined

In [ ]:
# Rule for high-low, bar for open-close
high_low = alt.Chart(df_clean).mark_rule().encode(
    x='date',
    y='low',
    y2='high'
)

open_close = alt.Chart(df_clean).mark_bar().encode(
    x='date',
    y='open',
    y2='close',
    color=alt.condition('datum.close > datum.open', alt.value('green'), alt.value('red'))
)

candlestick = high_low + open_close


In [ ]:
candlestick

In [ ]:
# In Polars before converting
df_clean = df_clean.with_columns([
    ((pl.col("close") - pl.col("close").mean()) / pl.col("close").std()).alias("z_score")
])

# In Altair
z_line = alt.Chart(df_clean).mark_line().encode(
    x='date',
    y='z_score'
)

thresholds = alt.Chart(pl.DataFrame({
    'z': [2, -2]
})).mark_rule(color='orange', strokeDash=[4, 4]).encode(
    y='z'
)

z_chart = z_line + thresholds


In [ ]:
z_chart

In [ ]:
dashboard = alt.hconcat(
    price_chart,
    volume_chart,
    trend_chart
).resolve_scale(
    x='shared'
)

In [ ]:
dashboard